In [1]:
import sys
sys.path.append('..')

from misc import load, load_class_labels

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"
val_restored_dir = dataset_root + "val_set_restored/"
val_preprocessed_dir = dataset_root + "val_set_preprocessed/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train_info_cleaned.csv', 1, sample_rate)
test_info = load(dataset_root + 'labels/test.csv')
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

2024-01-15 17:22:45.958016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 17:22:45.958071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 17:22:46.012833: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 17:22:46.125022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 17:22:47.160590: W tensorflow/compiler/tf2

In [2]:
from cnn_classification import build_finetune_network, train_network
from tensorflow import keras
from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input

save_path = "./tuned_models/ViTb16_noise_extended"

In [ ]:
extension = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(251, activation='softmax')])

loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy', keras.metrics.TopKCategoricalAccuracy(3)]

cut_layer = 'ExtractToken'
non_trainable_cut_layer = 'Transformer/encoderblock_11'

extended_model = build_finetune_network(lambda: vit.vit_b16(pretrained=True), extension, cut_layer,
                                        optimizer, metrics, loss, non_trainable_cut_layer=non_trainable_cut_layer)

In [ ]:
train_loss, train_accuracy, val_loss, val_accuracy = train_network(extended_model,
                                                                   save_path,
                                                                   train_info,
                                                                   val_info,
                                                                   train_dir,
                                                                   epochs=10,
                                                                   augment=True,
                                                                   strong_augment=False,
                                                                   batch_size=64,
                                                                   preprocess_input=preprocess_input)

In [3]:
from tensorflow import keras
from cnn_classification import evaluate_model, predict
from vit_keras import vit

model = keras.models.load_model(save_path)

/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2024-01-15 17:22:54.097662: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-15 17:22:54.293139: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA nod

In [4]:
predictions, names = predict(model, val_dir, 128, preprocess_input=vit.preprocess_inputs)

2024-01-15 17:23:20.375933: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2024-01-15 17:23:20.521478: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2024-01-15 17:23:23.692051: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


1/4 [======>.......................] - ETA: 0s

2024-01-15 17:23:30.721196: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.
2024-01-15 17:23:30.929946: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.


1/4 [======>.......................] - ETA: 0s

2024-01-15 17:23:34.058843: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 77070336 exceeds 10% of free system memory.


3/3 [==============================] - 5s 829ms/step


In [5]:
import numpy as np


ordered_indices = np.argsort(names)
predictions = np.array(predictions)
ordered_names = np.argsort(test_image_names)

predictions_logit = np.argmax(predictions[ordered_indices], axis=1)

filtered_info = test_info[test_info['filename'].isin(names)]
filtered_info.sort_values(by='filename', inplace=True)
labels = filtered_info['label'].values

ordered_y = labels

In [6]:
from sklearn.metrics import classification_report

print(classification_report(ordered_y, predictions_logit, target_names=class_labels))

                               precision    recall  f1-score   support

                      macaron       0.82      0.89      0.85        55
                      beignet       0.70      0.90      0.79        61
                      cruller       0.73      0.70      0.71        53
                  cockle_food       0.76      0.57      0.65        51
                       samosa       0.62      0.83      0.71        41
                     tiramisu       0.61      0.65      0.63        52
                      tostada       0.73      0.53      0.61        57
                     moussaka       0.53      0.61      0.57        54
                     dumpling       0.60      0.61      0.61        49
                      sashimi       0.63      0.78      0.70        46
                        knish       0.67      0.50      0.57        48
                    croquette       0.48      0.51      0.49        47
                     couscous       0.27      0.87      0.41        45
     

/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [7]:
import os
import pandas as pd
os.makedirs("predictions", exist_ok=True)

pred_file = "predictions/ViT_extended_standard"
np.save(pred_file, predictions)
df = pd.DataFrame({
    'filename': names
})
df.to_csv(pred_file + "_names.csv")

In [7]:
def top_k(k = 1):
    ohe = keras.utils.to_categorical(ordered_y, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    ordered_pred = predictions[ordered_indices]
    m.update_state(ohe, ordered_pred)
    return m.result().numpy()

In [8]:
top_k(1)

0.6098049

In [9]:
top_k(3)

0.78464234